# TAREA 3
# NLP2
# https://github.com/TextMiningNLP2/TAREA3/

In [22]:
# SE IMPORTAN LAS LIBRERIAS INSTALAS
# PREVIAMENTE SE HAN INSTALADO CIERTAS LIBERÍAS EN EL VENV

import enum
import pickle
from pprint import pprint
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import random
from typing import Generator, Dict, List, Tuple
from gensim.corpora import Dictionary
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import spacy
from spacy.lang.en import English
import nltk


In [24]:
# De ser necesario, descargar en Corpora / Stop_words
# Si se desea se puede descargar all_corpora pero toma tiempo
# nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [36]:
# CLASE DEFINIDORA DE SENTIMIENTOS
# obtenida del ejemplo prepare_data

class Sentiments(enum.Enum):
    POS = 'POS'
    NEG = 'NEG'

# FUNCION PARA DIVIDIR DATA
# obtenida del ejemplo train

def split_data(data: List, weights: List = (0.7, 0.15, 0.15)):
    split = {
        'train': [],
        'test': [],
        'validation': [],
    }
    for word in data:
        subset = random.choices(['train', 'test', 'validation'], weights=weights)[0]
        split[subset].append(word)

    return split

# FUNCION CONVERTIR ORACIONES A PALABRAS
# obtenida del ejemplo tokenize

def sentences_to_words(sentences: List[str]) -> List[List[str]]:
    words = []
    for sentence in sentences:
        # https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess
        words.append(simple_preprocess(str(sentence), deacc=True))  # deacc=True elimina la puntuación
    return words

# FUNCION QUE REMUEVE LOS STOPWORDS DADOS
# obtenida del ejemplo tokenize

def remove_stopwords(documents: List[List[str]]) -> List[List[str]]:
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords.words('english')]
            for doc in documents]

# FUNCION PARA APRENDER BIGRAMAS
# obtenida del ejemplo tokenize

def learn_bigrams(documents: List[List[str]]) -> List[List[str]]:
    # We learn bigrams
    #  https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.Phrases
    bigram = Phrases(documents, min_count=5, threshold=10)

    # we reduce the bigram model to its minimal functionality
    bigram_mod = Phraser(bigram)

    # we apply the bigram model to our documents
    return bigram_mod

# FUNCION PARA CREAR BIGRAMAS
# obtenida del ejemplo tokenize

def create_bigrams(bigram_model, documents: List[List[str]]):
    return [bigram_model[doc] for doc in documents]
    

In [13]:
df = pd.read_csv('../data/raw/reviews.csv')

df['rating'] = df['rating'].astype(dtype='int64')
df['sentiment'] = df['rating'].apply(lambda x: Sentiments.POS if x >= 40 else Sentiments.NEG)

data_classes = {sentiment.value: df[df['sentiment'] == sentiment]['review'].values.tolist() for sentiment in Sentiments}

pos_rev = data_classes['POS']
neg_rev = data_classes['NEG']

splitPOS = split_data(pos_rev)
splitNEG = split_data(neg_rev)


In [26]:
# split_data(pos_rev)['train'][0]

In [27]:
# pprint(data_classes['NEG'])

In [21]:
# PARA GUARDAR ARCHIVO BINARIO DE PYTHON
# NO ES NECESARIO CORRER, YA ESTA GUARDADA UNA COPIA

with open(r"../data/interim/review_classes.pkl", "wb") as output_file:
    pickle.dump(data_classes, output_file)

In [25]:
# PARA LEER ARCHIVO BINARIO DE PYTHON
# CORRER ESTE PARA LEER LA COPIA GUARDADA

with open(r"../data/interim/review_classes.pkl", "rb") as input_file:
    review_classes = pickle.load(input_file)

In [33]:
# SE DIVIDEN LAS CLASES EN POSITIVAS Y NEGATIVAS

positive_reviews = review_classes['POS']
negative_reviews = review_classes['NEG']

In [34]:
# SE DIVIDEN LAS CLASES EN PALABRAS UNICAS
# NO ES NECESARIO CORRER
# ABAJO LO TENEMOS PROCESADO, SOLO HACER ELCTURA

# print(negative_reviews[1])
negative_words = sentences_to_words(negative_reviews)
positive_words = sentences_to_words(positive_reviews)
# print(negative_words[1])

In [39]:
# PARA GUARDADO DE NEGATIVE/POSITIVE WORDS
# NO ES NECESARIO CORRER, YA QUE SE ENTREGAN LOS ARCHIVOS YA GUARDADOS
# SALTAR A LA SIGUIENTE CELDA
with open(r"../data/interim/negative_words.pkl", "wb") as output_file:
    pickle.dump(data_classes, output_file)
    
with open(r"../data/interim/positive_words.pkl", "wb") as output_file:
    pickle.dump(data_classes, output_file)

In [40]:
# PARA LECTURA DE LOS NEGATIVE/POSITIVE WORDS YA PROCESADO
# No se gasta tiempo en procesarlo otra vez

with open(r"../data/interim/review_classes.pkl", "rb") as input_file:
    negative_words = pickle.load(input_file)

with open(r"../data/interim/review_classes.pkl", "rb") as input_file:
    positive_words = pickle.load(input_file)

In [35]:
# DE LAS VARIABLES ANTERIORES, SE ELIMINAN LOS STOPWORDS

# print(negative_words[1])
negative_words_wo_stopwords = remove_stopwords(negative_words)
# print(negative_words_wo_stopwords[1])
positive_words_wo_stopwords = remove_stopwords(positive_words)

In [ ]:
# SE CREA MODELO DE BIGRAMAS
# A partir de TODO el corpus procesado
bigram_model = learn_bigrams(negative_words_wo_stopwords + positive_words_wo_stopwords)
negative_words_wo_stopwords_bigrams = create_bigrams(bigram_model, negative_words_wo_stopwords)